In [46]:
import pickle
window=5
final_data1 = pickle.load(open('/home/jupyter/mullah/Test/data_yield/data/data_sobc_'+str(window)+'_secs.v1.p','rb'))
final_data2 = pickle.load(open('/home/jupyter/mullah/Test/data_yield/data/data_sobc_'+str(window)+'_secs.p','rb'))

In [47]:
final_data = []
for i,a in enumerate(final_data2):
    a.append(final_data1[i][-1])
    final_data.append(a)
pickle.dump(final_data,open('/home/jupyter/mullah/Test/data_yield/data/data_sobc_'+str(window)+'_secs.with.ecg.p','wb'))

In [56]:
total = 0
for i,a in enumerate(final_data):
    if i%2!=0:
        continue
    ppg_data = a[-3]
    total+= (ppg_data[-1,0]-ppg_data[0,0])/(1000*3600)

In [57]:
total

65.94038539672852

In [41]:
import numpy as np
mins = []
for a in final_data:
    if len(a[-1])==0:
        continue
    mins.append(np.min(a[-1][:,0]))

In [42]:
min_value = np.min(mins)

In [43]:
sum_ = 0
for i,a in enumerate(final_data):
    if i%2!=0 or len(a[-1])==0:
        continue
    ecg_rr = a[-1]
    sum_+=ecg_rr.shape[0]

In [44]:
sum_/3600

39.341388888888886

In [58]:
import pickle 
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from sklearn.preprocessing import RobustScaler,StandardScaler, MinMaxScaler,QuantileTransformer
import pickle
from scipy.stats import skew,kurtosis,iqr
# from ecg import ecg_feature_computation
import math
# from hrvanalysis import remove_ectopic_beats
from joblib import Parallel, delayed
import warnings
warnings.filterwarnings('ignore')
from sklearn.decomposition import PCA
from pprint import pprint
from sklearn.metrics import f1_score
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score,accuracy_score,auc,classification_report,make_scorer,roc_curve
import itertools
from sklearn.model_selection import ParameterGrid, cross_val_predict, GroupKFold,GridSearchCV
from sklearn import preprocessing,metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from joblib import Parallel,delayed
delta = 0.1
from sklearn.metrics import roc_curve,auc,make_scorer
from copy import deepcopy

def my_score_auc(y_true,y_pred):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    return auc(fpr,tpr)

def f1Bias_scorer_CV(y_true,y_pred, ret_bias=False):
    probs = y_true
    y = y_pred
    if not ret_bias:
        fpr, tpr, thresholds = roc_curve(y_true, y_pred)
        return auc(fpr,tpr)
    precision, recall, thresholds = metrics.precision_recall_curve(y, probs)
    
    f1 = 0.0
    for i in range(0, len(thresholds)):
        if not (precision[i] == 0 and recall[i] == 0) and (precision[i]>=recall[i]-.05):
            f = 2 * (precision[i] * recall[i]) / (precision[i] + recall[i])
            if f > f1:
                f1 = f
                bias = thresholds[i]

    if ret_bias:
        return f1, bias
    else:
        return f1


def fit_model(X,y,groups,k,paramGrid):
    my_score = make_scorer(my_score_auc,needs_threshold=True)
    X = np.delete(X,k,axis=1)
    clf = Pipeline([('svc', SVC())])
    gkf = GroupKFold(n_splits=len(np.unique(groups)))
    grid_search = GridSearchCV(clf, paramGrid, n_jobs=10,cv=list(gkf.split(X,y,groups=groups)),
                               scoring='f1',verbose=1)
    grid_search.fit(X,y)
    clf = grid_search.best_estimator_
    clf.set_params(svc__probability=True)
    probs = cross_val_predict(clf,X,y,cv=gkf.split(X,y,groups=groups),n_jobs=20,method='predict_proba')[:,1]
    pp = deepcopy(probs)
    a,b = f1Bias_scorer_CV(probs, y, ret_bias=True)
    return np.array([a,k]).reshape(-1)


def get_results_backward_elimination(X,y,groups):
    my_score = make_scorer(my_score_auc,needs_threshold=True)
    delta = 0.1
    paramGrid = {'svc__kernel': ['rbf'],
             'svc__C': [1,10,100],
             'svc__gamma': [np.power(2,np.float(x)) for x in np.arange(-8, -2, .25)],
             'svc__class_weight': [{0: w, 1: 1 - w} for w in [.2,.3,.4,.5]],
             'svc__probability':[False]
    }
    feature_names = ['var','iqr','mean','median','80th','20th','heartrate','vlf','lf','hf','lf-hf']
#     gg = fit_model(deepcopy(X),y,groups,k,paramGrid)
    data = []
    clf = Pipeline([('svc', SVC())])
    gkf = GroupKFold(n_splits=len(np.unique(groups)))
    grid_search = GridSearchCV(clf, paramGrid, n_jobs=-1,cv=list(gkf.split(X,y,groups=groups)),
                               scoring='f1',verbose=1)
    grid_search.fit(X,y)
    clf = grid_search.best_estimator_
    clf.set_params(svc__probability=True)
    probs = cross_val_predict(clf,X,y,cv=gkf.split(X,y,groups=groups),n_jobs=20,method='predict_proba')[:,1]
    pp = deepcopy(probs)
    a,b = f1Bias_scorer_CV(probs, y, ret_bias=True)
    data.append(['all',a])
    print(data)
    while len(feature_names)>1:
        results = Parallel(n_jobs=30,verbose=4)(delayed(fit_model)(deepcopy(X),y,groups,k,paramGrid) for k,name in enumerate(feature_names))
        results = np.array(results)
        print(results,results.shape)
        ind_min = np.argmax(results[:,0])
        min_f1 = results[ind_min,0]
        min_index = np.int64(results[ind_min,1])
        name_feature = feature_names[min_index]
        data.append([name_feature,min_f1])
        X = np.delete(X,min_index,axis=1)
        feature_names = feature_names[:min_index] + feature_names[(min_index+1):]
        print(data)
    print(data)
    return data    
    
    

def get_f1(X,y,groups):
    my_score = make_scorer(my_score_auc,needs_threshold=True)
    paramGrid = {
             'svc__kernel': ['rbf'],
             'svc__C': [1,10,100],
             'svc__gamma': [np.power(2,np.float(x)) for x in np.arange(-8, -2, .5)],
             'svc__class_weight': [{0: w, 1: 1 - w} for w in [.2,.3,.4,.5]],
             'svc__probability':[False]
            }
    my_score = make_scorer(f1Bias_scorer_CV,needs_proba=True)
    clf = Pipeline([('svc',SVC())])
    gkf = GroupKFold(n_splits=len(np.unique(groups)))
    grid_search = GridSearchCV(clf, paramGrid, n_jobs=-1,cv=list(gkf.split(X,y,groups=groups)),
                               scoring='f1',verbose=2)
    grid_search.fit(X[:,:],y)
    clf = grid_search.best_estimator_
    clf.set_params(svc__probability=True)
    probs = cross_val_predict(clf,X,y,cv=gkf.split(X,y,groups=groups),n_jobs=20,method='predict_proba')[:,1]
    f1,bias = f1Bias_scorer_CV(probs, y, ret_bias=True)
    print(f1,bias,'bias')
    y_pred = probs
    y_pred[y_pred>=bias] = 1
    y_pred[y_pred<bias] = 0
    y_pred = np.int64(y_pred)
    clf.fit(X,y)
    return confusion_matrix(y,y_pred),f1_score(y,y_pred),precision_score(y,y_pred),recall_score(y,y_pred),clf,y,y_pred
    
def get_label(user_data,st,et):
    label = 2
    for k in range(user_data.shape[0]):
        if st>=user_data[k,0] and et<=user_data[k,1]:
            label = user_data[k,2]

    return label

def get_quality_features(a):
    feature = [np.mean(a),np.percentile(a,50),np.min(a)]
    return np.array(feature)

import numpy as np
from scipy import interpolate, signal
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
import matplotlib.patches as mpatches
from collections import OrderedDict

def frequencyDomain(RRints,tmStamps, band_type = None, lf_bw = 0.11, hf_bw = 0.1, plot = 0):
    NNs = RRints
    tss = tmStamps
    frequency_range = np.linspace(0.001, 1, 10000)
    NNs = np.array(NNs)
    NNs = NNs - np.mean(NNs)
    result = signal.lombscargle(tss, NNs, frequency_range)
        
    #Pwelch w/ zero pad     
    fxx = frequency_range 
    pxx = result 
    
    vlf= (0.003, 0.04)
    lf = (0.04, 0.15)
    hf = (0.15, 0.4)
    
    plot_labels = ['VLF', 'LF', 'HF']
        
    if band_type == 'adapted':     
            
        vlf_peak = fxx[np.where(pxx == np.max(pxx[np.logical_and(fxx >= vlf[0], fxx < vlf[1])]))[0][0]] 
        lf_peak = fxx[np.where(pxx == np.max(pxx[np.logical_and(fxx >= lf[0], fxx < lf[1])]))[0][0]]
        hf_peak = fxx[np.where(pxx == np.max(pxx[np.logical_and(fxx >= hf[0], fxx < hf[1])]))[0][0]]
    
        peak_freqs =  (vlf_peak, lf_peak, hf_peak) 
            
        hf = (peak_freqs[2] - hf_bw/2, peak_freqs[2] + hf_bw/2)
        lf = (peak_freqs[1] - lf_bw/2, peak_freqs[1] + lf_bw/2)   
        vlf = (0.003, lf[0])
        
        if lf[0] < 0:
            print('***Warning***: Adapted LF band lower bound spills into negative frequency range')
            print('Lower thresold of LF band has been set to zero')
            print('Adjust LF and HF bandwidths accordingly')
            lf = (0, lf[1])        
            vlf = (0, 0)
        elif hf[0] < 0:
            print('***Warning***: Adapted HF band lower bound spills into negative frequency range')
            print('Lower thresold of HF band has been set to zero')
            print('Adjust LF and HF bandwidths accordingly')
            hf = (0, hf[1])        
            lf = (0, 0)        
            vlf = (0, 0)
            
        plot_labels = ['Adapted_VLF', 'Adapted_LF', 'Adapted_HF']

    df = fxx[1] - fxx[0]
    vlf_power = np.trapz(pxx[np.logical_and(fxx >= vlf[0], fxx < vlf[1])], dx = df)      
    lf_power = np.trapz(pxx[np.logical_and(fxx >= lf[0], fxx < lf[1])], dx = df)            
    hf_power = np.trapz(pxx[np.logical_and(fxx >= hf[0], fxx < hf[1])], dx = df)             
    totalPower = vlf_power + lf_power + hf_power
    
    #Normalize and take log
    vlf_NU_log = np.log((vlf_power / (totalPower - vlf_power)) + 1)
    lf_NU_log = np.log((lf_power / (totalPower - vlf_power)) + 1)
    hf_NU_log = np.log((hf_power / (totalPower - vlf_power)) + 1)
    lfhfRation_log = np.log((lf_power / hf_power) + 1)   
    
    freqDomainFeats = {'VLF_Power': vlf_NU_log, 'LF_Power': lf_NU_log,
                       'HF_Power': hf_NU_log, 'LF/HF': lfhfRation_log}
    return freqDomainFeats


def weighted_avg_and_std(values, weights):
    """
    Return the weighted average and standard deviation.

    values, weights -- Numpy ndarrays with the same shape.
    """
    average = np.average(values, weights=weights)
    # Fast and numerically precise:
    variance = np.average((values-average)**2, weights=weights)
    return average, math.sqrt(variance)

def get_weighted_rr_features(a):
    a = np.repeat(a[:,0],np.int64(np.round(100*a[:,1])))
#     a = a[:,0]
    return np.array([np.var(a),iqr(a),np.mean(a),np.median(a),np.percentile(a,80),np.percentile(a,20),60000/np.median(a)])



no_of_feature = 11
from scipy.stats import variation
def combine_data_sobc(feature_matrix,user_col,label_col,quality_col,heart_rate_final,label_data,ecg_rr,qual=0,indicator='ppg'):
    if len(user_col)==0:
        return np.zeros((0,no_of_feature)),[],[],[],[],np.zeros((0,4))
    if indicator not in ['ppg']:
        try:
            heart_rate_final = np.zeros((ecg_rr.shape[0],3))
            heart_rate_final[:,0] = ecg_rr[:,0]
            heart_rate_final[:,1] = ecg_rr[:,1]
            heart_rate_final[:,2] = 1
        except:
            return np.zeros((0,no_of_feature)),[],[],[],[],np.zeros((0,4))
    participant = user_col[0]
    feature_matrix = []
    feature_matrix_quality = []
    user_col = []
    label_col = []
    quality_col = []
    if len(heart_rate_final)<80:
        return np.zeros((0,no_of_feature)),[],[],[],[],np.zeros((0,4))
    if len(heart_rate_final)<80:
        return np.zeros((0,no_of_feature)),[],[],[],[],np.zeros((0,4))
    ts_array = np.arange(min_value,heart_rate_final[-1,0],60000)
    ts_array = [t for t in ts_array if t>=heart_rate_final[0,0]]
    for t in ts_array:
        index = np.where((heart_rate_final[:,0]>=t-30000)&(heart_rate_final[:,0]<t+30000))[0]
        if len(index)<30:
            continue
        heart_rate_window = heart_rate_final[index]
        label = get_label(label_data,t-30000,t+30000)
        if len(heart_rate_window)<30:
            continue
        feature = get_weighted_rr_features(heart_rate_window[:,1:])
        feature_freq = frequencyDomain(heart_rate_window[:,1]/1000,heart_rate_window[:,0]/1000)
        feature = list(feature)+list(feature_freq.values())
        feature_quality = get_quality_features(heart_rate_window[:,2])
        if feature_quality[0]<qual:
            continue
        feature_matrix.append(np.array(feature).reshape(-1,no_of_feature))
        feature_matrix_quality.append(np.array(feature_quality).reshape(-1,3))
        user_col.append(participant)
        label_col.append(label)
        quality_col.append(t)
    return np.array(feature_matrix).reshape(-1,no_of_feature),user_col,label_col,quality_col,heart_rate_final,np.array(feature_matrix_quality).reshape(-1,3)

from sklearn import linear_model
def get_only_stress_no_stress(X,groups,y,X_qual,time_matching):
    y = np.int64(y)
    index = np.where(y<2)[0]
    X,groups,y,X_qual,time_matching = X[index,:],groups[index],y[index],X_qual[index],time_matching[index]
    ind = []
    for grp in np.unique(groups):
        tmp = np.where(groups==grp)[0]
        if len(np.unique(y[tmp]))>1 and np.sum(y[tmp])/len(y[tmp])>=.1 and len(y[tmp])>20:
            ind.extend(list(tmp))
    ind = np.int64(np.array(ind))
    return X[ind],y[ind],groups[ind],X_qual[ind],time_matching[ind]

def get_XY(final_data,window,qual1=0,indicator='ppg'):
    
    duration = window
    final_output = Parallel(n_jobs=30,verbose=1)(delayed(combine_data_sobc)(*(a+[qual1,indicator])) for i,a in enumerate(final_data) if i%2==0)
    X = np.zeros((0,no_of_feature))
    X_qual = []
    y = []
    groups = []
    time_matching = []
    for m in final_output:
        feature_matrix,user_col,label_col,quality_col,hr,quals = m
        if len(feature_matrix)<10:
            continue
        quals1 = np.array([1]*feature_matrix.shape[0])
        ss = np.repeat(feature_matrix[:,2],np.int64(np.round(100*quals1)))
        rr_70th = np.percentile(ss,20)
        rr_95th = np.percentile(ss,95)
        index = np.where((feature_matrix[:,2]>rr_70th)&(feature_matrix[:,2]<rr_95th))[0]
        for i in range(feature_matrix.shape[1]):
            m,s = weighted_avg_and_std(feature_matrix[index,i], quals1[index])
            feature_matrix[:,i]  = (feature_matrix[:,i] - m)/s
        tmp = StandardScaler().fit_transform(np.nan_to_num(feature_matrix))
        X = np.concatenate((X,feature_matrix))
        X_qual.extend(quals.reshape(1,-2,3))
        y.extend(label_col)
        groups.extend(user_col)
        time_matching.extend(list(quality_col))
    print(X.shape)
    time_matching = np.array(time_matching)
    y = np.array(y)
    groups = np.array(groups)
    X_qual = np.concatenate(X_qual)
    print(X_qual.shape,len(time_matching),qual1)
    y = y[~np.isnan(X).any(axis=1)]
    time_matching = time_matching[~np.isnan(X).any(axis=1)]
    groups = groups[~np.isnan(X).any(axis=1)]
    X_qual = X_qual[~np.isnan(X).any(axis=1)]
    X = X[~np.isnan(X).any(axis=1)]
    X,y,groups,X_qual,time_matching = get_only_stress_no_stress(X,groups,y,X_qual,time_matching)
    print(X.shape,y.shape,len(np.unique(groups)),X_qual.shape,time_matching.shape)
    return X,y,groups,X_qual,time_matching

no_of_feature = 11
# feature_index = np.array([0,1,3,4,5,6,8])
for q in [.2]:
#     X,y,groups,X_qual,X_time = get_XY(final_data,window=5,qual1=q,indicator='ecg')
#     m,f,p,r,clf,y,y_pred = get_f1(X,y,groups)
#     pickle.dump([X,y,groups,X_qual,X_time,clf],open('ecg_data.p','wb'))
    X,y,groups,X_qual,X_time = get_XY(final_data,window=5,qual1=q,indicator='ppg')
    m,f,p,r,clf,y,y_pred = get_f1(X,y,groups)
    print(m,q,f,p,r)
#     pickle.dump([X,y,groups,X_qual,X_time,clf],open('ppg_data.p','wb'))
    
#     print(q,m,f,p,r)
#     results.append([m,f,p,r,clf])

[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done  14 out of  36 | elapsed:    8.1s remaining:   12.7s


(2701, 11)
(2701, 3) 2701 0.2
(1452, 11) (1452,) 28 (1452, 3) (1452,)


[Parallel(n_jobs=30)]: Done  36 out of  36 | elapsed:   14.1s finished


Fitting 28 folds for each of 144 candidates, totalling 4032 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 269 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 552 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 917 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 1362 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 1889 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 2496 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done 3185 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-1)]: Done 4032 out of 4032 | elapsed:   13.8s finished


0.7170370370370371 0.375473876664173 bias
[[1019  102]
 [  89  242]] 0.2 0.7170370370370371 0.7034883720930233 0.7311178247734139


In [7]:
[.62,.62,.62,.61,.64,.61,.64,.66,.70,.71]
[.65,.65,.65,.65,.67,.64,.67,.68,.72,.73]
[.64,.65,.65,.65,.66,.70,.73,.75,.79,.81]
[.697,.697,.70,.699,.72,.73,.77,.77,.79,.81]
[0,.05,.1,.15,.2,.25,.3,.35,.4,.45]
import pickle
from sklearn.decomposition import PCA
from pprint import pprint
from sklearn.metrics import f1_score
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score,accuracy_score,auc,classification_report,make_scorer,roc_curve
import itertools
from sklearn.model_selection import ParameterGrid, cross_val_predict, GroupKFold,GridSearchCV
from sklearn import preprocessing,metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from joblib import Parallel,delayed
import pandas as pd
delta = 0.1
from sklearn.metrics import roc_curve,auc,make_scorer
def get_dataframe(filename,type_of_stress='stress_ppg_weighted'):
    X,y,groups,X_qual,X_time,clf = pickle.load(open(filename,'rb'))
    gkf = GroupKFold(n_splits=len(np.unique(groups)))
    probs = cross_val_predict(clf,X,y,cv=gkf.split(X,y,groups=groups),n_jobs=20,method='predict_proba')[:,1]
    data = pd.DataFrame({'time':X_time,'user':groups,'stress_likelihood':probs,'groundtruth':y,'quality_features':list(X_qual)})
    data['type'] = type_of_stress
    return data

In [8]:
data_ppg_not_weighted = get_dataframe('ppg_data_not_weighted.p',type_of_stress='stress_ppg_not_weighted')
data_ppg = get_dataframe('ppg_data.p',type_of_stress='stress_ppg_weighted')
data_ecg = get_dataframe('ecg_data.p',type_of_stress='stress_ecg')

In [9]:
data_ppg = pd.concat([data_ppg,data_ppg_not_weighted])

In [10]:
data_col = []
for user in data_ppg.user.unique():
    tmp_ppg = data_ppg[data_ppg.user==user]
    tmp_ecg = data_ecg[data_ecg.user==user]
    tmp_ecg['stress_likelihood_ecg'] = tmp_ecg['stress_likelihood']
    tmp_ppg = pd.merge(tmp_ppg, tmp_ecg[['stress_likelihood_ecg','time']], how='inner',on=['time'])
    data_col.append(tmp_ppg)

In [11]:
data_all = pd.concat(data_col)

In [12]:
# f1Bias_scorer_CV(data_all['groundtruth'],data_all['stress_likelihood'])

In [13]:
data_all.time.unique().shape,data_all.shape,data_all.dropna().shape,data_ecg.shape,data_ppg.shape

((1310,), (2620, 7), (2620, 7), (1481, 6), (3552, 6))

In [14]:
# data_all.dropna()[-200:]
# 7096+2956

In [15]:
from scipy.stats import pearsonr
df_weighted = [df for i,df in data_all.groupby('type',as_index=False)][1]
pearsonr(df_weighted['stress_likelihood'],df_weighted['stress_likelihood_ecg'])

(0.7796307510615962, 3.872171784974957e-268)

In [16]:
from scipy.stats import pearsonr
df_weighted = [df for i,df in data_all.groupby('type',as_index=False)][0]
pearsonr(df_weighted['stress_likelihood'],df_weighted['stress_likelihood_ecg'])

(0.7316869545323773, 5.883671307762137e-220)

In [17]:
data_all['min'] = data_all['quality_features'].apply(lambda a:a[2])
data_all['median'] = data_all['quality_features'].apply(lambda a:a[1])
data_all['mean'] = data_all['quality_features'].apply(lambda a:a[0])

In [18]:
quality_feature_names = ['mean']
quality_range = np.arange(0,.85,.05)
def get_corr(df):
    all_data = []
    type_of_stress,user = df['type'].values[0],df['user'].values[0]
    for i,qname in enumerate(quality_feature_names):
        for q in quality_range:
            tmp_df = df[df[qname]>=q]
            if tmp_df.shape[0]<5:
                continue
            corr = pearsonr(tmp_df['stress_likelihood_ecg'],tmp_df['stress_likelihood'])[0]
            all_data.append([type_of_stress,user,qname,q,corr])
    return pd.DataFrame(all_data,columns=['type','user','qualityname','qualityvalue','corr'])
data_corr = data_all.groupby(['user','type'],as_index=False).apply(get_corr)

In [20]:
feature_names = ['corr']

def get_mean_std(df):
    t = df['qualityname'].values[0]
    v = '$\ge$'+str(df['qualityvalue'].values[0])
    tt = df['type'].values[0]
    means = []
    for f in feature_names:
        m = np.median(df[f])
        s = np.std(df[f])/1.7
        value = "{:.2f}".format(m)+"$\pm$"+"{:.2f}".format(s)
        means.append(value)
    return pd.DataFrame([[tt,t,v]+means],columns=['type','qualitytype','qualityvalue']+feature_names)
# final_corr.groupby(['Quality Type','Quality Value'],as_index=False).apply(get_mean_std).reset_index(drop=True).to_csv('field_features.csv')

data_corr.groupby(['qualityvalue','type','qualityname'],as_index=False).apply(get_mean_std)

,,type,qualitytype,qualityvalue,corr
0,0,stress_ppg_not_weighted,mean,$\ge$0.0,0.78$\pm$0.19
1,0,stress_ppg_weighted,mean,$\ge$0.0,0.85$\pm$0.17
2,0,stress_ppg_not_weighted,mean,$\ge$0.05,0.78$\pm$0.19
3,0,stress_ppg_weighted,mean,$\ge$0.05,0.85$\pm$0.17
4,0,stress_ppg_not_weighted,mean,$\ge$0.1,0.78$\pm$0.19
5,0,stress_ppg_weighted,mean,$\ge$0.1,0.85$\pm$0.17
6,0,stress_ppg_not_weighted,mean,$\ge$0.15000000000000002,0.78$\pm$0.19
7,0,stress_ppg_weighted,mean,$\ge$0.15000000000000002,0.83$\pm$0.17
8,0,stress_ppg_not_weighted,mean,$\ge$0.2,0.79$\pm$0.18
9,0,stress_ppg_weighted,mean,$\ge$0.2,0.85$\pm$0.16


In [198]:
data_corr.head()

type                                  user  \
0 0  stress_ppg_not_weighted  063b1819-aaa3-4368-8430-6ce1a665c1d3   
  1  stress_ppg_not_weighted  063b1819-aaa3-4368-8430-6ce1a665c1d3   
  2  stress_ppg_not_weighted  063b1819-aaa3-4368-8430-6ce1a665c1d3   
  3  stress_ppg_not_weighted  063b1819-aaa3-4368-8430-6ce1a665c1d3   
  4  stress_ppg_not_weighted  063b1819-aaa3-4368-8430-6ce1a665c1d3   

    qualityname  qualityvalue      corr  
0 0        mean          0.00  0.581240  
  1        mean          0.05  0.581240  
  2        mean          0.10  0.581240  
  3        mean          0.15  0.571741  
  4        mean          0.20  0.571741